In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import base64
import os
import re
import time

def save_base64_image(data, file_path):
    """Base64形式の画像をデコードして保存する関数"""
    try:
        image_data = base64.b64decode(data)
        with open(file_path, "wb") as f:
            f.write(image_data)
        print(f"保存成功: {file_path}")
    except Exception as e:
        print(f"エラー: {e}")

def download_image(url, file_path):
    """URLから画像をダウンロードして保存する関数"""
    try:
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(r.content)
            print(f"保存成功: {file_path}")
    except Exception as e:
        print(f"エラー: {e}")

def get_imgs(system):
    # ご自分の環境に合わせて任意のディレクトリパスを指定してください。
    base_save_dir = r"C:\Users\NDF05\OneDrive\ドキュメント\VsCodeProject\Pytyon\torend\Select_imegMnager\Slect_imgs"
    save_dir = os.path.join(base_save_dir, system)
    os.makedirs(save_dir, exist_ok=True)

    base64_string = "data:image/jpeg;base64,"
    png_base64_string = "data:image/png;base64,"


    # 検索クエリとURLの生成
    query = system +" 服 imagesize:1080x1080"
    url = "https://www.google.com/search?q={}&hl=ja&tbm=isch".format(query)

    # Chromeドライバの設定
    service = Service(executable_path='path_to_chromedriver')  # chromedriverのパスを指定してください
    driver = webdriver.Chrome()

    driver.get(url)

    # 緩やかにスクロールするJavaScript関数を定義
    scroll_js = """
    function slowScroll() {
        var scrollHeight = document.body.scrollHeight;
        var currentScroll = 0;
        var scrollIncrement = 10;  // 小さな増分でスクロールする（ピクセル）
        
        function scroll() {
            if (currentScroll < 100000) {
                window.scrollBy(0, scrollIncrement);
                currentScroll += scrollIncrement;
                console.log(currentScroll);  // 現在のスクロール位置をコンソールに出力
                setTimeout(scroll, 50);  // 50ミリ秒ごとにスクロールする
            }
        }
        
        scroll();  // スクロール関数を実行
    }
    slowScroll();

    """

    # スクロールを実行
    driver.execute_script(scroll_js)
    time.sleep(60*2)  # ページが読み込まれるまで待つ

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    # ベースURLの取得
    base_url = "https://www.google.com"

    img_tags = soup.find_all("img")
    img_urls = []

    for img_tag in img_tags:
        url = img_tag.get("src")
        if url is None:
            url = img_tag.get("data-src")
        if url is not None:
            if not url.startswith("http"):
                url = urljoin(base_url, url)
            img_urls.append(url)
            print(url)

    # 画像を保存
    for index, url in enumerate(img_urls):
        file_name = "{}.jpg".format(index)
        image_path = os.path.join(save_dir, file_name)

        if base64_string in url:
            url = url.replace(base64_string, "")
            save_base64_image(data=url, file_path=image_path)
        elif png_base64_string in url:
            url = url.replace(png_base64_string, "")
            save_base64_image(data=url, file_path=image_path)
        else:
            download_image(url=url, file_path=image_path)

    driver.quit()

    
casual_styles = [
    "ベーシック",
    "アメリカンカジュアル（アメカジ）",
    "モードカジュアル",
    "スポーツカジュアル",
    "ストリートカジュアル",
    "アウトドア"
]

elegant_styles = [
    "エレガンス",
    "ガーリー",
    "オフィスカジュアル",
    "キャリア",
    "マニッシュ",
    "デザイナーズ"
]

other_styles = [
    "トラッド",
    "ノームコア",
    "ナチュラル",
    "韓国（オルチャン）",
    "量産型",
    "地雷系"
]

sum_list = casual_styles + elegant_styles + other_styles

for i in range(len(sum_list)):
    get_imgs(sum_list[i])
    time.sleep(2)